# Задание 2. Итерационные методы решения линейных систем


In [16]:
from latexifier import latexify
from IPython.display import display, Markdown, Latex
import numpy as np
import pandas as pd
from typing import Callable, List, Tuple, Union, Any
import warnings
warnings.filterwarnings('ignore')


In [54]:
class Matrix(np.ndarray):
    """extends np.ndarray"""

    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        return obj

    def inverse_rows(self, i: int, j: int):
        """Swaps i-th and j-th rows of a matrix"""
        self[i], self[j] = self[j].copy(), self[i].copy()

    def inverse_cols(self, i: int, j: int):
        """Swaps i-th and j-th columns of a matrix"""
        self = self.transpose()
        self.inverse_rows(i, j)
        self = self.transpose()

    def tex(self) -> str:
        """Latexifies the matrix rounding up values up to 6 decimal values"""
        return latexify(self)

    def norm(self) -> float:
        """Calculates Matrix norm using `np.linalg.norm` with parameter `np.inf`"""
        return np.linalg.norm(self, np.inf)

    def inv(self) -> Any:
        return Matrix(np.linalg.inv(self))


def append(M: np.ndarray, N: np.ndarray) -> Matrix:
    """Appends two numpy arrays with axis=1"""
    return Matrix(np.append(arr=np.asarray(M), values=N, axis=1))


In [8]:
def Gauss(A: Matrix, B: Matrix, n: int) -> Matrix:
    """
    Вычисляет решения линейного уравнения Ax = B методом Гаусса с выбором
    главного элемента по столбцам (с перестановкой строк)
    """
    AB = append(A, B)
    X = [[0] for _ in range(n)]
    for k in range(n):
        j = k
        for i in range(k+1, n):
            if abs(AB[j][k]) < abs(AB[i][k]):
                j = i
        AB.inverse_rows(k, j)
        for j in range(k+1, n):
            c = AB[j][k] / AB[k][k]
            for i in range(n+1):
                AB[j][i] -= c * AB[k][i]
    X[n-1][0] = AB[n-1][n] / AB[n-1][n-1]
    for k in range(n-1, -1, -1):
        s = 0
        for i in range(k+1, n):
            s += AB[k][i] * X[i][0]
        X[k][0] = (AB[k][n] - s) / AB[k][k]
    return Matrix(np.array(X))


In [95]:
def print_tex(*argv) -> None:
    """Displays a LaTex Markdown output"""
    res = ' '.join(['$$'] + [(latexify(arg) if isinstance(arg,
                   np.ndarray) else str(arg)) for arg in argv] + ['$$'])
    display(Markdown(res))


def get_latex_column(arg: str, n: int) -> str:
    """
    Prepares a string with LaTex syntax for a column of `arg` with indeces form 1 to `n`
    """
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = arg + "_{" + str(i+1) + "}" + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)


def get_exact_column(col: Matrix, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of {col} elements"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = str(float(col[i][0])) + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)


def print_gauss_output(A: Matrix, B: Matrix, X: Matrix, n: int, name: str) -> None:
    """Oupputs the result of solving a linear equation"""
    B_ = A @ X
    print_tex(r'\text{', name, '}~X^* = ',
              get_latex_column("x^*", n), '=', get_exact_column(X, n))
    print_tex(rf'A \times X^* = ', A.tex(), X.tex(), '=', B_.tex(), '= B^*')
    print_tex('B = ', get_exact_column(B, n),
              r'\stackrel{?}{=}', get_exact_column(B_, n), '= B^*')
    print_tex(r'B - B^* = ', get_exact_column(B - B_, n))


## Вариант 12


In [120]:
n = 4
A = Matrix(np.random.rand(n, n))
B = Matrix(np.random.rand(n, 1))
for i in range(n):
    A[i][i] += np.sum([A[i][j] for j in range(n) if i != j])
print_tex('A=', A, ',~~', 'B=', B)


$$ A= \begin{pmatrix} 0.587864 & 0.100389 & 0.223769 & 0.220756 \\ 0.877197 & 2.245134 & 0.451472 & 0.785147 \\ 0.613065 & 0.451668 & 1.948599 & 0.118462 \\ 0.179308 & 0.499202 & 0.963665 & 2.235260 \end{pmatrix} ,~~ B= \begin{pmatrix} 0.493136 \\ 0.445390 \\ 0.778082 \\ 0.528644 \end{pmatrix} $$

Найдем решение $x^*$ методом Гаусса.


In [121]:
X_Gauss = Gauss(A, B, n)
print_gauss_output(A, B, X_Gauss, n, 'Решение методом Гаусса:')


$$ \text{ Решение методом Гаусса: }~X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix}
0.744568846481772\\
-0.1782564217484637\\
0.19839972141909132\\
0.13105068823793647
\end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 0.587864 & 0.100389 & 0.223769 & 0.220756 \\ 0.877197 & 2.245134 & 0.451472 & 0.785147 \\ 0.613065 & 0.451668 & 1.948599 & 0.118462 \\ 0.179308 & 0.499202 & 0.963665 & 2.235260 \end{pmatrix} \begin{pmatrix} 0.744569 \\ -0.178256 \\ 0.198400 \\ 0.131051 \end{pmatrix} = \begin{pmatrix} 0.493136 \\ 0.445390 \\ 0.778082 \\ 0.528644 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
0.49313640203906206\\
0.4453902473695439\\
0.7780823154154177\\
0.528644328079071
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
0.49313640203906206\\
0.4453902473695438\\
0.7780823154154176\\
0.5286443280790711
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
0.0\\
1.1102230246251565e-16\\
1.1102230246251565e-16\\
-1.1102230246251565e-16
\end{pmatrix} $$

In [134]:
def compare_result(name: str, val: Matrix, n: int = n) -> None:
    print_tex(name, '=', get_exact_column(val, n), r'\stackrel{?}{=}', get_exact_column(X_Gauss, n), '=X^*')
    print_tex(rf'||X^* - {name}||_\infty=', (X_Gauss - val).norm())

Преобразуем исходную систему к системе вида $x = H_Dx + g_D$
$$ H_D = E - D^{-1}A $$
$$ g_D = D^{-1}b $$


In [122]:
D, D_inv = Matrix(np.zeros((n, n))), Matrix(np.zeros((n, n)))

for i in range(n):
    D[i][i] = A[i][i]
    D_inv[i][i] = 1 / A[i][i]

H_D = Matrix(np.identity(n) - D_inv @ A)
g_D = Matrix(D_inv @ B)
print_tex('H_D=', H_D, ',~~', 'g_D=', g_D)
print_tex(rf'||H_D||_\infty = ', H_D.norm())


$$ H_D= \begin{pmatrix} 0 & -0.170768 & -0.380648 & -0.375522 \\ -0.390710 & 0 & -0.201089 & -0.349710 \\ -0.314618 & -0.231791 & 0 & -0.060793 \\ -0.080218 & -0.223331 & -0.431120 & 0 \end{pmatrix} ,~~ g_D= \begin{pmatrix} 0.838861 \\ 0.198380 \\ 0.399303 \\ 0.236502 \end{pmatrix} $$

$$ ||H_D||_\infty =  0.9415099293846889 $$

Найдем априорную оценку для того $k$, при котором $||x^* - x^k||_\infty < \varepsilon,~\varepsilon = 0.001$


In [123]:
def apr_eval(
    H: Matrix = H_D,
    g: Matrix = g_D,
    k: int = 1
) -> float:
    return H.norm()**k / (1 - H.norm()) * g.norm()


eps = 1.e-3
k = 1
while (apr_eval(k=k) >= eps):
    k += 1
k


159

In [135]:
def iterative_method(
    H: Matrix = H_D,
    G: Matrix = g_D,
    n: int = n
) -> Union[Matrix, Matrix, int]:
    next = Matrix(np.zeros((n, 1)))
    iter = 0
    while (X_Gauss - next).norm() >= eps:
        prev, next = next, H @ next + G
        iter += 1
    return next, prev, iter


iter_ans, prev, iter = iterative_method()
# print_tex(r'\text{Решение методом простой итерации:}~',
#           rf'X^{({iter})}=', get_exact_column(iter_ans, n))
# print_tex(rf'||X^* - X^{({iter})}||_\infty=', (X_Gauss - iter_ans).norm())
compare_result(rf'X^{({iter})}', iter_ans)


$$ X^{24} = \begin{pmatrix}
0.7437106507542685\\
-0.17916767980576995\\
0.19773382167652925\\
0.13034031291014622
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
0.744568846481772\\
-0.1782564217484637\\
0.19839972141909132\\
0.13105068823793647
\end{pmatrix} =X^* $$

$$ ||X^* - X^{24}||_\infty= 0.0009112580573062412 $$

In [125]:
print('Априорная оценка:', apr_eval(k=iter))


Априорная оценка: 3.3760250526350646


In [126]:
def apostr_eval(
    x_k: Matrix = iter_ans,
    x_prev: Matrix = prev,
    H: Matrix = H_D,
) -> float:
    return H.norm() / (1 - H.norm()) * (x_k - x_prev).norm()


print('Апостериорная оценка:', apostr_eval(iter_ans, prev))


Апостериорная оценка: 0.033294499575683284


In [137]:
def spec_radius(H: Matrix = H_D):
    return max(np.absolute(np.linalg.eigvals(H)))

rho = spec_radius(H_D)

def luster_approx(x_k: Matrix = iter_ans, x_prev: Matrix = prev, rho: float = rho) -> Matrix:
    return prev + (x_k - x_prev) / (1 - rho)

X_luster = luster_approx()

compare_result(rf'X^{({iter})}_\text{{lust}}', X_luster)


$$ X^{24}_\text{lust} = \begin{pmatrix}
0.7364908287384513\\
-0.18683390422230925\\
0.19213174612129377\\
0.12436407350591616
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
0.744568846481772\\
-0.1782564217484637\\
0.19839972141909132\\
0.13105068823793647
\end{pmatrix} =X^* $$

$$ ||X^* - X^{24}_\text{lust}||_\infty= 0.008577482473845544 $$